# 🔌 Notebook 3 — Gateway + X‑API‑KEY Security (Phase 1, hands‑on)

> **Goal:** place the **Gateway** in front of `tasks-service`, validate routing `/api/tasks/**`, and enable **optional X‑API‑KEY** security.  
> **No Eureka/Config** at this phase. Stack is **on‑prem/local** (Docker Compose via `run_compose.sh`) with an **STS** option as well.

---

## 0) What is an **API Gateway** in cloud context (and what we implement here)

### 0.1 Definition (cloud)
An **API Gateway** is the **single north‑south entry point** for a system’s APIs. It receives client requests and can **apply policies** (authentication, authorization, CORS, rate‑limit), perform **transformations** (headers/path/payload), handle **versioning**, and **route** each call to the right service. It also exposes **observability** (health/metrics) and helps with API governance.

> In public clouds we often use **managed services** (e.g., **AWS API Gateway**, **Azure API Management**).  
> In this course, for dev/on‑prem, we implement a lightweight gateway using **Spring Cloud Gateway (SCG)**.

### 0.2 What we are doing **now** (Phase 1, with Spring Cloud Gateway)
- **Single entry point** at `http://localhost:8080` with a route:
  - `/api/tasks/**` → forwards to `tasks-service` (port **8081**).
- **Minimal security (optional)**:
  - If the environment variable **`API_KEY`** is defined for the gateway process, clients **must** send header **`X-API-KEY: <value>`**; otherwise the gateway returns **401**.
  - If **`API_KEY`** is **not** defined, the gateway runs in **open mode** (no header required) for dev/demos.
- **Observability**:
  - Gateway **health** at `/actuator/health` (UP/DOWN).

🔗 **12‑Factor notes touched here:** **#7 Port Binding**, **#3 Config**, **#5 Build/Release/Run**, **#6 Processes**, **#10 Dev/Prod parity**, **#11 Logs**, **#9 Disposability**.

### 0.3 Practical difference: **API Gateway vs Load Balancer** (and close relatives)

| Component | Purpose | Layer | API policies (auth, rate‑limit, keys) | Transformations (headers/path/payload) | API‑level observability | Cloud examples |
|---|---|---|---|---|---|---|
| **API Gateway** | Client **entry point** (north‑south); publish & govern APIs | L7 | **Yes** (rich) | **Yes** (rich) | **Yes** (API‑centric) | **AWS API Gateway**, **Azure API Management** |
| **Load Balancer** | Distribute traffic to instances | L4/L7 | Limited/No | Limited | Basic | **AWS ALB/NLB**, **Azure App Gateway/Load Balancer** |
| **Ingress** (k8s) | Expose HTTP/HTTPS services | L7 | Depends on controller | Some (rules) | Basic | Nginx/Traefik ingress |
| **Service Mesh** | **East‑west** traffic between services | L7 (data plane) | mTLS/network policies (between services) | Not the focus | Metrics/tracing **between services** | Istio/Linkerd |

> **Here**, our **gateway** is an **application‑level** layer (SCG) applying **simple policies** (X‑API‑KEY) and **routing**.  
> A typical **load balancer** mostly decides **which instance** receives traffic and **does not** implement rich **API** policies by itself.

### 0.4 Mapping this to **AWS/Azure** (forward‑looking note, not implemented now)
- **AWS** (example path):
  - **AWS API Gateway** as public entry (API policies, keys/JWT, rate‑limit)  
  - Targeting an **ALB** or directly **ECS/EKS** (service/container running `tasks-service`)  
  - Secrets in **AWS Secrets Manager**, identities via **Cognito** (if we evolve to JWT).
- **Azure** (example path):
  - **Azure API Management (APIM)** as public entry (policies, keys/JWT, dev portal optional)  
  - Targeting **Azure Application Gateway**/**AKS**/**Container Apps** hosting `tasks-service`  
  - Secrets in **Key Vault**, identity via **Microsoft Entra ID**.

The idea: what SCG does for us now (**route + X‑API‑KEY**) has **managed** counterparts in clouds.  
For this course, Phase 1 = **SCG on‑prem/local**, simple and didactic.

---

## 🎯 What you will achieve

- Run **gateway (8080)** + **tasks-service (8081)** using **Docker Compose** (**recommended**) or **STS**.  
- Validate **routing**: `GET/POST/PATCH` via `http://localhost:8080/api/tasks/**`.  
- **Enable** and **test** **X‑API‑KEY** (header required when the env var is defined).  
- Check **observability** for **both services**: `/actuator/health`.

---

## 🧭 Theory→Practice bridge (12 Factors in this Phase 1)

| Practical action | Factors touched |
|---|---|
| Run two services (gateway + backend) | **#1 Codebase**, **#5 Build/Release/Run**, **#6 Processes** |
| Route `/api/**` in the gateway | **#7 Port Binding** |
| API key via environment variable | **#3 Config**, **#10 Dev/Prod parity** |
| Health for both services | **#9 Disposability**, **#7 Port Binding** |
| Compose for local integration | **#10 Dev/Prod parity**, **#5 Build/Release/Run** |
| Separate logs per service | **#11 Logs** |

---

## 🧰 Prerequisites

- **Java 21**, **Maven**, **Git**  
- **Docker** + **Docker Compose** (for `run_compose.sh`)  
- **STS (Spring Tool Suite)** (optional alternative to run locally)  
- **Postman** (recommended) — alternative: `curl`

---

## 1) Get the code for **Phase 1**

```bash
git clone https://github.com/smartlearningci/cloud_java
cd cloud_java
git checkout phase-1
```

**Included in this phase:**  
- `tasks-service` (port 8081, H2, Actuator)  
- `gateway` (port 8080) with route `/api/tasks/**` → `tasks-service`  
- **Optional X‑API‑KEY** (enforced **only** if `API_KEY` exists in the gateway’s environment)

🔗 **12‑Factor #1 Codebase** — phase **tags** make it easy to navigate stable states and keep the cohort aligned.

---

## 2) Bring up the stack with **Docker Compose** (recommended)

### 2.1 (Optional) Enable X‑API‑KEY
Choose **ONE** method:

- **A — session environment variable**
  ```bash
  export API_KEY=supersecret
  ```
- **B — `.env` file** at the project root
  ```
  API_KEY=supersecret
  ```

> Without `API_KEY`, the gateway **does not** require `X-API-KEY` (open mode for dev/demos).

### 2.2 Start services
```bash
./run_compose.sh
# or:
# docker compose up --build
```

### 2.3 Verify health
- Gateway: `http://localhost:8080/actuator/health`  
- Backend: `http://localhost:8081/actuator/health` (expect **UP** for both)

🔗 **12‑Factor #5, #7, #9, #10** — Compose approximates real multi‑service setups; each app exposes HTTP; fast start/stop; config from env.

---

## 3) (Alternative) Run via **STS**

### 3.1 `tasks-service` (8081)
- Import Maven project → main class → **Run As → Spring Boot App**.

### 3.2 `gateway` (8080)
- Import the gateway project → (to require API key) set `API_KEY` in the Run Configuration → **Run As → Spring Boot App**.

🔗 **12‑Factor #3 Config, #5 Build/Release/Run** — flipping env (`API_KEY` on/off) changes behavior **without rebuild**.

---

## 4) Test via **Gateway** with **Postman**

### 4.1 Environment
- `baseUrl = http://localhost:8080/api`  
- `apiKey = supersecret` (if you enabled `API_KEY`)

### 4.2 Requests

1) **List**
- **GET** `{{baseUrl}}/tasks`  
- If `API_KEY` is enabled: header `X-API-KEY: {{apiKey}}`  
- Expected: `200 OK` (without header when enabled → **401**)

2) **Create**
- **POST** `{{baseUrl}}/tasks`  
  Headers: `Content-Type: application/json` (+ `X-API-KEY` if enabled)  
  Body:
```json
{
  "title": "Task via Gateway",
  "description": "Notebook 3 - Phase 1",
  "projectId": "GW",
  "assignee": "Bruno"
}
```
- Expected: `201 Created` + Task

3) **Update status**
- **PATCH** `{{baseUrl}}/tasks/{{taskId}}/status`  
  Headers: `Content-Type: application/json` (+ `X-API-KEY` if enabled)  
  Body:
```json
{ "status": "DOING" }
```
- Expected: `200 OK`

4) **Get by ID**
- **GET** `{{baseUrl}}/tasks/{{taskId}}` (+ `X-API-KEY` if enabled)  
- Expected: `200 OK` (or **404** if the id does not exist)

🔗 **12‑Factor #7 Port Binding, #11 Logs** — single entry `/api/**` simplifies clients; gateway and backend logs remain **separate and clear**.

---

## 5) Quick tests with **curl** (terminal)

> **Without API_KEY**:
```bash
curl -s http://localhost:8080/api/tasks | jq
```

> **With API_KEY** (e.g., `supersecret`):
```bash
# missing header → 401 Unauthorized
curl -i http://localhost:8080/api/tasks

# with header → 200 OK
curl -s -H "X-API-KEY: supersecret" http://localhost:8080/api/tasks | jq

# create
curl -s -H "X-API-KEY: supersecret" -H "Content-Type: application/json"   -d '{"title":"Task via Gateway","description":"Notebook 3 - Phase 1","projectId":"GW","assignee":"Bruno"}'   http://localhost:8080/api/tasks | jq
```

---

## 6) Observability (Gateway + Backend)

- **Gateway**: `GET http://localhost:8080/actuator/health` → `{"status":"UP"}`  
- **Tasks-service**: `GET http://localhost:8081/actuator/health` → `{"status":"UP"}`

If the gateway returns 502/503, check whether the backend is **UP** and inspect logs.

🔗 **12‑Factor #9 Disposability** — simple health reduces MTTR and aids local orchestration.

---

## 7) Expected errors

- **401 Unauthorized (gateway)**: `API_KEY` enabled and `X-API-KEY` missing or wrong.  
- **404 Not Found (backend via gateway)**: non‑existent resource/wrong route.  
- **409/422 (backend)**: invalid state transition (per service policy).  
- **502/503 (gateway)**: backend unavailable.

🔗 **12‑Factor #10 Dev/Prod parity** — consistent status codes/payloads improve predictability across environments.

---

## 8) Troubleshooting

- **8080/8081 busy**: stop old processes or change ports in the local profile.  
- **API_KEY not applied**: ensure `.env` is correct or `export API_KEY=...` **before** `up`; in STS, set it in **Run Configuration**.  
- **502 on gateway**: validate `http://localhost:8081/actuator/health`.  
- **Postman**: don’t forget `X-API-KEY` when enabled; use `Content-Type: application/json` for `POST/PATCH`.

🔗 **12‑Factor #3 Config, #5 Build/Release/Run** — the same binary/image changes behavior **only** via environment.

---

## ✅ Notebook 3 checkpoints

- [ ] I brought up **gateway (8080)** + **tasks-service (8081)** via **Compose** or **STS**.  
- [ ] I validated **/actuator/health** on **both** services (**UP**).  
- [ ] I confirmed **routing** `/api/tasks/**` works.  
- [ ] I enabled the **API key** (optional) and observed **401** without header and **200/201** with correct `X-API-KEY`.  
- [ ] I repeated **POST/GET/PATCH/GET{id}** always **via the gateway**.

---

## 🔜 Next

**Notebook 4 — On‑prem Deploy with Docker Compose + STS**  
- Consolidate on‑prem execution with `run_compose.sh`.  
- Tips for remote exposure (port/TLS optional) and multi‑arch (Raspberry Pi, if used later).
